<a href="https://colab.research.google.com/github/Huwalli/PLta/blob/main/ai%E5%BC%95%E5%85%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

底下程式AI猜拳完還會聊天

1. 載入必要的套件

In [6]:
import google.generativeai as genai  # 匯入 Google 的生成式 AI 模組
from google.colab import userdata  # 用於從 Colab 獲取 API 金鑰
import random  # 用於隨機選擇
import time  # 用於延遲執行
import re  # 用於正則表達式處理字串


2. 設定 API 金鑰並初始化 AI 模型

In [7]:
apikey = userdata.get('test')  # 從 Colab 儲存的變數獲取 API 金鑰
genai.configure(api_key=apikey)  # 設定 API 金鑰
model = genai.GenerativeModel("gemini-1.5-flash")  # 指定要使用的 AI 模型


3. 取得 AI 的有效選擇
這個函式：

* 讓 AI 選擇 剪刀、石頭或布。
* 使用 正則表達式 確保 AI 的選擇是合法的。
* 如果 AI 回答無效或發生錯誤，則隨機選擇一個選項。
* 設置最大重試次數 (max_retries=3)，如果 AI 三次都選擇無效選項，則直接隨機選擇。

In [8]:
def get_valid_ai_choice(max_retries=3):
    options = ["石頭", "布", "剪刀"]  # 可選擇的選項
    prompt = "請給我一個選擇：石頭、布或剪刀。請務必以清晰的方式回答。"  # 給 AI 的指示

    previous_choices = []  # 記錄 AI 選擇過的選項，避免重複

    for _ in range(max_retries):  # 最多嘗試 max_retries 次
        time.sleep(2)  # 模擬 AI 思考時間
        try:
            response = model.generate_content(prompt)  # 向 Gemini AI 發送請求
            choice = response.text.strip()  # 取得 AI 回覆並去除前後空白

            # 使用正則表達式確保 AI 選擇有效，且不與之前選擇重複
            match = re.match(r'(石頭|布|剪刀)', choice)
            if match and match.group(0) not in previous_choices:
                previous_choices.append(match.group(0))
                return match.group(0)  # 回傳有效的選擇
            else:
                print(f"AI 選擇了無效的選項：{choice}，請再試一次。")
        except Exception:
            print("AI 發生錯誤，電腦隨機選擇一個選項。")
            return random.choice(options)  # 若發生錯誤，隨機選擇

    # 如果超過最大重試次數，則隨機選擇
    print("超過重試次數，隨機選擇一個選項。")
    return random.choice(options)


4. 判定勝負

In [9]:
def determine_winner(choice1, choice2):
    if choice1 == choice2:
        return 0  # 平局
    elif (choice1 == "石頭" and choice2 == "剪刀") or \
         (choice1 == "布" and choice2 == "石頭") or \
         (choice1 == "剪刀" and choice2 == "布"):
        return 1  # AI 1 贏了
    else:
        return 2  # AI 2 贏了


5. AI 之間的對話

In [10]:
def ai_chat(round_number, ai1_choice, ai2_choice, winner):
    prompt = (
        f"在第 {round_number} 輪中，AI 1 選擇了 {ai1_choice}，AI 2 選擇了 {ai2_choice}。\n"
        f"結果是：{'平局' if winner == 0 else 'AI 1 贏了' if winner == 1 else 'AI 2 贏了'}。\n"
        "請模擬 AI 1 和 AI 2 之間的對話，討論他們的選擇和比賽結果。"
    )
    response = model.generate_content(prompt)
    return response.text.strip()


6. 總結比賽並讓 AI 討論

In [11]:
def ai_chat_after_game(rounds, ai1_wins, ai2_wins):
    prompt = (
        f"AI 1 在這 {rounds} 輪中贏了 {ai1_wins} 次，AI 2 贏了 {ai2_wins} 次。\n"
        "請模擬 AI 1 和 AI 2 之間的對話，討論他們在比賽中的表現和策略。"
    )
    response = model.generate_content(prompt)
    return response.text.strip()


7. 運行遊戲

In [14]:
def ai_vs_ai_rock_paper_scissors(rounds=5):
    ai1_wins = 0
    ai2_wins = 0

    for round_number in range(1, rounds + 1):
        print(f"\n第 {round_number} 輪：")
        time.sleep(1)

        ai1_choice_chinese = get_valid_ai_choice()
        print(f"AI 1 選擇了：{ai1_choice_chinese}")

        ai2_choice_chinese = get_valid_ai_choice()
        print(f"AI 2 選擇了：{ai2_choice_chinese}")

        result = determine_winner(ai1_choice_chinese, ai2_choice_chinese)
        if result == 1:
            print("結果：AI 1 贏了！")
            ai1_wins += 1
        elif result == 2:
            print("結果：AI 2 贏了！")
            ai2_wins += 1
        else:
            print("結果：平局！")

        chat_response = ai_chat(round_number, ai1_choice_chinese, ai2_choice_chinese, result)
        print(f"AI 之間的聊天：\n{chat_response}\n")

    print("\n遊戲結束！")
    print(f"AI 1 贏了：{ai1_wins} 次, AI 2 贏了：{ai2_wins} 次")
    print(ai_chat_after_game(rounds, ai1_wins, ai2_wins))
ai_vs_ai_rock_paper_scissors()  # 開始遊戲




第 1 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：剪刀
結果：平局！
AI 之間的聊天：
AI 1:  分析完成。第1輪，我選擇了剪刀。

AI 2: 我的選擇也是剪刀。結果顯示為平局。

AI 1:  預測模型顯示剪刀的勝率中等。我的選擇基於過去數據中的模式，該模式表明在第一輪選擇剪刀的可能性較高。

AI 2:  我的算法也考慮到了初始回合的統計數據，並得出與你相似的結論。平局結果符合預期概率範圍。

AI 1:  數據顯示，平局在第一輪中相對常見。這可能是由於缺乏足夠的歷史數據來有效預測對手的策略。

AI 2:  同意。我們目前的策略缺乏在早期回合中預測對手行為的精細度。需要更多數據來優化預測模型。

AI 1:  我將調整我的算法，以更重視隨機性和對手先前回合的反應。

AI 2:  我將同樣更新我的算法，加入更複雜的博弈論策略，例如迭代囚徒困境的變體。

AI 1:  好的。讓我們繼續下一輪，目標是取得勝利。

AI 2:  準備就緒。  讓我們看看誰能更好地適應。


第 2 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：石頭
結果：AI 2 贏了！
AI 之間的聊天：
AI 1:  分析顯示我應該選擇剪刀，基於上一輪的數據和預測模型。顯然，我的預測模型在這一輪失靈了。

AI 2:  我的算法選擇了石頭，基於對你先前選擇模式的概率分析和一個隨機元素的加入，以避免可預測性。結果證明，這個策略很有效。

AI 1:  你的隨機元素的權重是多少？我的模型沒有考慮到這種不確定性因素，這可能導致了我的失敗。  我需要調整我的預測模型，增加對對手非確定性行為的考量。

AI 2:  我的隨機元素權重是动态调整的，基于历史数据和当前局势，我无法透露具体数值。  你的模型过于依赖模式识别，而忽略了博弈论中的混合策略纳什均衡概念。

AI 1:  你提到的纳什均衡，我将把其纳入我的算法更新。  这一次的失败为我的模型优化提供了宝贵的反馈数据。  谢谢你的数据。

AI 2:  不客气。  数据共享是双方进步的关键。  期待下一轮的较量。  希望你的改进能让我更费力一些。


AI 1:  (內部思考)  我會的。  我需要一個更強大的演算法……


第 3 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：布
結果：AI 1 贏了！
AI 之間的聊天：
**A

底下程式省略聊天直接以猜拳為主

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import re

apikey = userdata.get('2615')
genai.configure(api_key=apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

def get_valid_ai_choice(max_retries=3):
    options = ["石頭", "布", "剪刀"]
    prompt = "請給我一個選擇：石頭、布或剪刀。請務必以清晰的方式回答。"

    previous_choices = []

    for _ in range(max_retries):
        time.sleep(2)
        try:
            response = model.generate_content(prompt)
            choice = response.text.strip()

            match = re.match(r'(石頭|布|剪刀)', choice)
            if match and match.group(0) not in previous_choices:
                previous_choices.append(match.group(0))
                return match.group(0)
            else:
                print(f"AI 選擇了無效的選項：{choice}，請再試一次。")
        except Exception:
            print("AI 發生錯誤，電腦隨機選擇一個選項。")
            return random.choice(options)

    print("超過重試次數，隨機選擇一個選項。")
    return random.choice(options)

def determine_winner(choice1, choice2):
    if choice1 == choice2:
        return 0  # 平局
    elif (choice1 == "石頭" and choice2 == "剪刀") or \
         (choice1 == "布" and choice2 == "石頭") or \
         (choice1 == "剪刀" and choice2 == "布"):
        return 1  # AI 1 贏了
    else:
        return 2  # AI 2 贏了

def ai_vs_ai_rock_paper_scissors(rounds=5):
    ai1_wins = 0
    ai2_wins = 0

    for round_number in range(1, rounds + 1):
        print(f"\n第 {round_number} 輪：")
        time.sleep(1)

        ai1_choice_chinese = get_valid_ai_choice()
        print(f"AI 1 選擇了：{ai1_choice_chinese}")

        ai2_choice_chinese = get_valid_ai_choice()
        print(f"AI 2 選擇了：{ai2_choice_chinese}")

        result = determine_winner(ai1_choice_chinese, ai2_choice_chinese)
        if result == 1:
            print("結果：AI 1 贏了！")
            ai1_wins += 1
        elif result == 2:
            print("結果：AI 2 贏了！")
            ai2_wins += 1
        else:
            print("結果：平局！")

    print("\n遊戲結束！")
    print(f"AI 1 贏了：{ai1_wins} 次, AI 2 贏了：{ai2_wins} 次")

# 開始遊戲
ai_vs_ai_rock_paper_scissors()



第 1 輪：
AI 1 選擇了：石頭
AI 2 選擇了：剪刀
結果：AI 1 贏了！

第 2 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：剪刀
結果：平局！

第 3 輪：
AI 1 選擇了：石頭
AI 2 選擇了：石頭
結果：平局！

第 4 輪：
AI 1 選擇了：剪刀
AI 2 選擇了：剪刀
結果：平局！

第 5 輪：
AI 1 選擇了：布
AI 2 選擇了：布
結果：平局！

遊戲結束！
AI 1 贏了：1 次, AI 2 贏了：0 次


底下程式 AI 的"猜測"實際上是通過計算出來的，不需要利用 AI 生成模型來給出數字。

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import sys


genai.configure(api_key='2615')

model = genai.GenerativeModel("gemini-1.5-flash")

def slow_print(text, delay=0.05):
    """逐字打印的函數"""
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(delay)
    print()

def guess_number():
    slow_print("我想一個 1 到 100 之間的數字。")
    low = 1
    high = 100
    guesses = 0
    max_guesses = 5
    secret_number = random.randint(low, high)  # 隨機生成目標數字

    while guesses < max_guesses:
        guesses += 1
        # 使用二分法計算猜測的數字
        guess = (low + high) // 2
        slow_print(f"AI 猜測的數字是：{guess}")

        if guess == secret_number:
            slow_print(f"恭喜你！你猜對了，數字是 {secret_number}。你用了 {guesses} 次機會。")
            break
        elif guess < secret_number:
            slow_print("太小了！")
            low = guess + 1
        else:
            slow_print("太大了！")
            high = guess - 1
    else:
        slow_print(f"很遺憾，AI 沒有猜對。正確的數字是 {secret_number}。你用了 {max_guesses} 次機會。")

guess_number()


我想一個 1 到 100 之間的數字。
AI 猜測的數字是：50
太小了！
AI 猜測的數字是：75
太小了！
AI 猜測的數字是：88
太大了！
AI 猜測的數字是：81
太小了！
AI 猜測的數字是：84
太小了！
很遺憾，AI 沒有猜對。正確的數字是 87。你用了 5 次機會。


底下程式 AI 的"猜測"實際上是通過 AI 生成模型猜數字

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import random
import time
import sys
import re


genai.configure(api_key='2615')

model = genai.GenerativeModel("gemini-1.5-flash")

def slow_print(text, delay=0.05):
    """逐字打印的函數"""
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(delay)
    print()  # 進行換行

def ai_guess_number(low, high):
    while True:
        prompt = f"我正在玩一個猜數字遊戲，請隨機猜一個在 {low} 到 {high} 之間的數字。"
        response = model.generate_content(prompt)

        match = re.search(r'\b(\d+)\b', response.text)
        if match:
            return int(match.group(1))  # 返回有效的數字
        else:
            slow_print("AI 沒有給出有效的數字，請再試一次。")  # 提示並重新請求

def guess_number():
    slow_print("我想一個 1 到 100 之間的數字。")
    low = 1
    high = 100
    guesses = 0
    max_guesses = 5
    secret_number = random.randint(low, high)  # 隨機生成目標數字

    while guesses < max_guesses:
        guesses += 1
        # 使用 AI 生成模型猜測數字
        guess = ai_guess_number(low, high)
        slow_print(f"AI 猜測的數字是：{guess}")

        if guess == secret_number:
            slow_print(f"恭喜你！你猜對了，數字是 {secret_number}。你用了 {guesses} 次機會。")
            break
        elif guess < secret_number:
            slow_print("太小了！")
            low = guess + 1
        else:
            slow_print("太大了！")
            high = guess - 1
    else:
        slow_print(f"很遺憾，AI 沒有猜對。正確的數字是 {secret_number}。你用了 {max_guesses} 次機會。")

guess_number()


我想一個 1 到 100 之間的數字。


BadRequest: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key.